In [70]:
with open("the-verdict.txt", 'r') as f:
  raw_text = f.read()

In [71]:
print(f"Total number of characters: {len(raw_text)}")

Total number of characters: 20479


In [72]:
raw_text[:99]

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no '

In [69]:
!pip install tiktoken

import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

## **Input-Target Pairs**

In [73]:
enc_text = tokenizer.encode(raw_text)

In [74]:
enc_sample = enc_text[:50]

In [75]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

In [76]:
print(f"X: {x}")
print(f"y: {y}")

X: [40, 367, 2885, 1464]
y: [367, 2885, 1464, 1807]


In [77]:
for i in range(1, context_size + 1):
  context = enc_sample[:i]
  desired = enc_sample[i]

  print(f"Context: {context} ----> Desired: {desired}")

Context: [40] ----> Desired: 367
Context: [40, 367] ----> Desired: 2885
Context: [40, 367, 2885] ----> Desired: 1464
Context: [40, 367, 2885, 1464] ----> Desired: 1807


In [78]:
context_size = 10
for i in range(1, context_size + 1):
  context = enc_sample[:i]
  desired = enc_sample[i]

  print(f"{tokenizer.decode(context)} ---> {tokenizer.decode([desired])}")

I --->  H
I H ---> AD
I HAD --->  always
I HAD always --->  thought
I HAD always thought --->  Jack
I HAD always thought Jack --->  G
I HAD always thought Jack G ---> is
I HAD always thought Jack Gis ---> burn
I HAD always thought Jack Gisburn --->  rather
I HAD always thought Jack Gisburn rather --->  a


In [79]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetv1(Dataset):
  def __init__(self, text, max_length, stride, tokenizer):
    self.input_ids = []
    self.target_ids = []

    # Tokenize the entire text
    token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

    # Use sliding window to chunk the book into overlapping sequences of max_length
    for i in range(0, len(token_ids) - max_length, stride):
      input_chunks = token_ids[i: i + max_length]
      target_chunks = token_ids[i + 1: i + max_length + 1]
      self.input_ids.append(torch.tensor(input_chunks))
      self.target_ids.append(torch.tensor(target_chunks))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, ids):
    return self.input_ids[ids], self.target_ids[ids]

In [80]:
import os

def CreateDataLoader(text, batch_size = 4, max_length = 256, stride = 128, shuffle = True, drop_last = True, num_workers = os.cpu_count()):

  # Initialize the tokenizer
  tokenizer = tiktoken.get_encoding("gpt2")

  # Create Dataset
  dataset = GPTDatasetv1(text,  max_length, stride, tokenizer)

  # Create DataLoader
  dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = shuffle, drop_last = drop_last, num_workers = num_workers)

  return dataloader

In [91]:
dataloader = CreateDataLoader(raw_text, batch_size = 8, max_length = 4, stride = 1, shuffle = False)

data_iter = iter(dataloader)
input, target = next(data_iter)
print("Inputs:\n", input)
print("Target:\n", target)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257]])
Target:
 tensor([[  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257],
        [10899,  2138,   257,  7026]])
